In [1]:
import os
from glob import glob

import pandas as pd

In [2]:
def filter_df(df, criterion):
    df_ = df.copy(deep=True)
    for k, v in criterion.items():
        df_ = df_[df_[k] == v]
    return df_

def read_csvs(files):
    return pd.concat([pd.read_csv(fn, index_col=0) for fn in files], axis=0)

def parse_results(ssl, arch, pretrain, downstream, task_type, output_dir):
    # read csvs
    df = read_csvs(glob(os.path.join(output_dir, f'{ssl}_{arch}_{pretrain}_{downstream}_{task_type}_seed*.csv')))
    
    # parse results
    tables = []
    for method, criterion in MODELS.items():
        df_method = filter_df(df, criterion)
        for metric in ['pred_entropy-tau', 'brier-tau']:
            correlation = df_method[metric].mean()  # average across seeds
            tables.append([ssl, arch, pretrain, downstream, metric, method, correlation])
    
    df = pd.DataFrame(tables, columns=['SSL', 'ARCH', 'PRETRAIN', 'DOWNSTREAM', 'METRIC', 'METHOD', 'TAU'])
    df = df.sort_values(
        by=['SSL', 'ARCH', 'DOWNSTREAM', 'METRIC'], 
        ascending=True
    )
    
    display(df)

In [3]:
MODELS = {
    'NC-100': {
        'method': 'NC',
        'k': 100,
        'dist': 'cosine',
    },
    'DIST-1': {
        'method': 'ens-AvgDist',
        'k': 1,
        'dist': 'cosine',
        
    },
    'L2-NORM': {
        'method': 'ens-L2Norm',
        'dist': 'euclidean',
    },
    'FV': {
        'method': 'FV',
        'dist': 'cosine',
    },
}

# In-Distribution

In [4]:
parse_results(
    ssl = 'simclr',
    arch = 'resnet18',
    pretrain = 'cifar100',
    downstream = 'cifar100',
    task_type = 'binary',
    output_dir = './results',
)

,SSL,ARCH,PRETRAIN,DOWNSTREAM,METRIC,METHOD,TAU
1,simclr,resnet18,cifar100,cifar100,brier-tau,NC-100,0.259031
3,simclr,resnet18,cifar100,cifar100,brier-tau,DIST-1,0.063516
5,simclr,resnet18,cifar100,cifar100,brier-tau,L2-NORM,0.064928
7,simclr,resnet18,cifar100,cifar100,brier-tau,FV,-0.171034
0,simclr,resnet18,cifar100,cifar100,pred_entropy-tau,NC-100,0.312990
2,simclr,resnet18,cifar100,cifar100,pred_entropy-tau,DIST-1,0.075662
4,simclr,resnet18,cifar100,cifar100,pred_entropy-tau,L2-NORM,0.119843
6,simclr,resnet18,cifar100,cifar100,pred_entropy-tau,FV,-0.189135


# Transfer Learning

In [5]:
parse_results(
    ssl = 'simclr',
    arch = 'resnet18',
    pretrain = 'imagenet32',
    downstream = 'cifar100',
    task_type = 'binary',
    output_dir = './results',
)

,SSL,ARCH,PRETRAIN,DOWNSTREAM,METRIC,METHOD,TAU
1,simclr,resnet18,imagenet32,cifar100,brier-tau,NC-100,0.134267
3,simclr,resnet18,imagenet32,cifar100,brier-tau,DIST-1,-0.025093
5,simclr,resnet18,imagenet32,cifar100,brier-tau,L2-NORM,0.050689
7,simclr,resnet18,imagenet32,cifar100,brier-tau,FV,-0.166790
0,simclr,resnet18,imagenet32,cifar100,pred_entropy-tau,NC-100,0.168010
2,simclr,resnet18,imagenet32,cifar100,pred_entropy-tau,DIST-1,0.013778
4,simclr,resnet18,imagenet32,cifar100,pred_entropy-tau,L2-NORM,0.121969
6,simclr,resnet18,imagenet32,cifar100,pred_entropy-tau,FV,-0.165961
